In [1]:
%load_ext autoreload
%autoreload 2
!pwd
!hostname

/home/ajasja/projects/truncator
dig26


In [2]:
#structs = truncator.read_file_lines("/home/ajasja/struct/ZCON/nMS.txt", trim=True)
#for s in structs:
#    !cp /home/ajasja/struct/ZCON/{s}  /home/ajasja/struct/ZCON/nMS

In [3]:
import numpy as np
from glob import glob
import numpy as np
import os
import truncator
import pandas as pd

In [4]:
structs = glob("/home/ajasja/struct/ZCON/nMS/*.pdb")
structs;

In [5]:
#truncator.cut_bundles(struct_name=structs[0], out_dir='out/R1/cut/4H', num_heptads=4, )

In [6]:
for l in [3,3.5,4,4.5,5]:
    for struct in structs:
        try:
            truncator.cut_bundles(struct_name=struct, out_dir=f'out/R3/01_cut/{l:1.1f}H', num_heptads=l)
        except:
            print (struct, l)

 PyMOL not running, entering library mode (experimental)


/home/ajasja/projects/truncator/truncator/cutting.py:77: RuntimeWarning: divide by zero encountered in true_divide
  bottoms = np.arange(highest_bottom, lowest_top, step_delta_z)


/home/ajasja/struct/ZCON/nMS/ZCON_9_FL_CC_trim.pdb 3
/home/ajasja/struct/ZCON/nMS/ZCON_9_FL_CC_trim.pdb 3.5
/home/ajasja/struct/ZCON/nMS/ZCON_9_FL_CC_trim.pdb 4
/home/ajasja/struct/ZCON/nMS/ZCON_9_FL_CC_trim.pdb 4.5
/home/ajasja/struct/ZCON/nMS/ZCON_9_FL_CC_trim.pdb 5


In [5]:
%%time
for l in [4.5,5]:
    all_cut = glob(f"out/R3/01_cut/{l:1.1f}H/*.pdb")
    for struct in all_cut:
        try:
            truncator.regroup_chains(struct, f"out/R3/02_regruped/{l:1.1f}H",
                            new_chain_A="AB", new_chain_B="CD")
            truncator.regroup_chains(struct, f"out/R3/02_regruped/{l:1.1f}H",
                            new_chain_A="AD", new_chain_B="BC")
            truncator.regroup_chains(struct, f"out/R3/02_regruped/{l:1.1f}H",
                            new_chain_A="A", new_chain_B="BCD")
            truncator.regroup_chains(struct, f"out/R3/02_regruped/{l:1.1f}H",
                            new_chain_A="B", new_chain_B="ACD")
            truncator.regroup_chains(struct, f"out/R3/02_regruped/{l:1.1f}H",
                            new_chain_A="C", new_chain_B="ABD")
            truncator.regroup_chains(struct, f"out/R3/02_regruped/{l:1.1f}H",
                            new_chain_A="D", new_chain_B="ABC")
        except:
            print (struct)

 PyMOL not running, entering library mode (experimental)
out/R3/01_cut/4.5H/ZCON_39__numH4.5__from-18.55__to22.19.pdb
out/R3/01_cut/4.5H/ZCON_39__numH4.5__from-20.54__to20.20.pdb
CPU times: user 51.3 s, sys: 992 ms, total: 52.3 s
Wall time: 59.2 s


In [4]:
all_cut = glob(f"out/R3/02_regruped/4.0H/*.pdb")            
truncator.score_interface(all_cut[13],
                    script_name="truncator/xml/05_specific_interface_markers.xml",
                    out_dir="out/R3/03_score/4.0H", test_run=False, skip_existing=True)

/software/rosetta/latest/bin/rosetta_scripts     -out:file:o  /home/ajasja/projects/truncator/out/R3/03_score/4.0H/ZCON_112__numH4__from-20.48__to16.48__grA-BCD.pdb     -out:file:scorefile  /home/ajasja/projects/truncator/out/R3/03_score/4.0H/ZCON_112__numH4__from-20.48__to16.48__grA-BCD.sc       -parser:protocol truncator/xml/05_specific_interface_markers.xml -s out/R3/02_regruped/4.0H/ZCON_112__numH4__from-20.48__to16.48__grA-BCD.pdb     -print_pymol_selection 0     -beta     -in:file:fullatom     -renumber_pdb 1     -overwrite     -out:file:scorefile_format json     -holes:dalphaball truncator/scripts/DAlphaBall.gcc -out:no_nstruct_label     > /home/ajasja/projects/truncator/out/R3/03_score/4.0H/ZCON_112__numH4__from-20.48__to16.48__grA-BCD.log
Saving to: /home/ajasja/projects/truncator/out/R3/03_score/4.0H/ZCON_112__numH4__from-20.48__to16.48__grA-BCD_IA.sc


'/home/ajasja/projects/truncator/out/R3/03_score/4.0H/ZCON_112__numH4__from-20.48__to16.48__grA-BCD_IA.sc'

In [6]:
import dask
from dask.distributed import Client, progress
client = Client(scheduler_file='scheduler.json'); client

Client Scheduler: tcp://172.16.131.56:5981 Dashboard: http://172.16.131.56:8787/status,Cluster Workers: 2 Cores: 2 Memory: 4.00 GB


In [10]:

res = []
for l in [3,3.5,4,4.5,5]:
    all_cut = glob(f"out/R3/02_regruped/{l:1.1f}H/*AD-BC*.pdb")
    for struct in all_cut:
        res.append(client.submit(truncator.score_interface, struct,
                script_name="truncator/xml/05_specific_interface_markers.xml",
                out_dir=f"out/R3/03_score/{l:1.1f}H", test_run=False))

In [14]:
all_cut

['out/R3/02_regruped/5.0H/ZCON_13_110__numH5__from-17.52__to27.29__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_131__numH5__from-23.68__to17.62__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_114__numH5__from-16.80__to28.05__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_94_N3C1__numH5__from-19.21__to25.88__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_90__numH5__from-23.85__to21.80__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_13_110__numH5__from-15.55__to29.26__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_131__numH5__from-20.06__to21.25__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_94__numH5__from-21.54__to26.19__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_103__numH5__from-20.86__to19.90__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_15_re_loop__numH5__from-16.52__to27.55__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_110__numH5__from-24.44__to22.86__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_114__numH5__from-22.72__to22.13__grAD-BC.pdb',
 'out/R3/02_regruped/5.0H/ZCON_15_re_loop__numH5_

distributed.utils - ERROR - can't start new thread
Traceback (most recent call last):
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/distributed/utils.py", line 648, in log_errors
    yield
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/distributed/client.py", line 873, in _reconnect
    yield self._ensure_connected()
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/distributed/client.py", line 888, in _ensure_connected
    connection_args=self.connection_args)
  File "/home/ajasja/.conda/envs/pyro_scripts/lib/python3.6/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/home/ajasja/.conda/envs

In [12]:
import dask
dask.distributed.wait(res, timeout=5);

TimeoutError: Timeout

In [16]:
retry_futures = []
for k, v in client.who_has().items():
    future = dask.distributed.Future(key=k, client=client, inform=True, state=None)
    if future.status == "error":
        retry_futures.append(future)
        

In [18]:
client.retry(retry_futures)

In [20]:
###Do reloop
def do_reloop(struct_name):
    reloop = truncator.reloop(struct_name, "out/R2/relooped/random", 
                          rosetta_bin="/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease",
                          script_name="truncator/xml/12_redesign_loops_tl_gen_profile_SymAnn.xml",
                            test_run=False, skip_existing=True)
    return reloop

In [22]:

structs = glob("out/R2/cut/3.0H/*.pdb"); structs
rstructs = np.random.choice(structs, 100); 
rstructs = sorted(set(rstructs)); 
rstructs;

In [23]:
res = client.map(do_reloop, rstructs)
results = client.gather(res)

In [24]:
lstructs = truncator.find_no_output_pdbs("out/R2/relooped/random", ignore_unfinished=True) 
all_struct = glob("out/R2/cut/*/*.pdb")
lstructs = [truncator.find_input(os.path.basename(struct), all_struct) for struct in lstructs]

In [25]:
import sys
def do_reloop_longer(struct_name):
    reloop = truncator.reloop(struct_name, "out/R2/relooped/random_longer_loops", 
                          rosetta_bin="/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease",
                          script_name="truncator/xml/13_redesign_loops_tl_gen_profile_SymAnn_longerloops.xml",
                            test_run=False, skip_existing=True)
    return reloop
    


In [26]:
res = client.map(do_reloop_longer, lstructs)
results = client.gather(res)

KilledWorker: ('do_reloop_longer-a3bea14f4fc1223abd43ce9f98c8fe84', 'tcp://172.16.131.59:29787')

In [ ]:
truncator.regroup_chains("testing_out//ZCON_37_3+1__numH4__from-22.25__to17.45.pdb", "testing_out/grouped",
                        new_chain_A="AB", new_chain_B="CD")

In [ ]:
truncator.regroup_chains("testing_out//ZCON_37_3+1__numH4__from-22.25__to17.45.pdb", "testing_out/grouped",
                        new_chain_A="C", new_chain_B="ABD", save_segi="True")

In [ ]:
truncator.regroup_chains("testing_out//ZCON_37_3+1__numH4__from-22.25__to17.45.pdb", "testing_out/grouped",
                        new_chain_A="C", new_chain_B="ABD", save_segi=False,
                        out_name="testing_out/grouped/ZCON_37_3+1__numH4__from-22.25__to17.45__grC-ABD.nosegi.pdb")

In [ ]:
#cmd.load("testing_out//ZCON_37_3+1__numH4__from-22.25__to17.45.pdb")
#m = cmd.get_model("chain A")

In [ ]:
pdb_files = glob("testing_out/grouped/*.pdb")
pdb_files

In [ ]:
%%time
sf= []
for pdb in pdb_files:
    sf.append(truncator.score_interface(pdb,
                    script_name="truncator/xml/03_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_quick_no_holes.xml",
                    out_dir="testing_out/grouped/scoring", test_run=False))
sf

In [ ]:
import truncator
score_files = sorted(glob("testing_out/grouped/*_IA.sc"))
score_files

In [ ]:
scores = truncator.read_score_files(score_files, skiprows=False, load_seq=False).set_index('decoy')
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores.transpose())

In [ ]:
score

In [ ]:
files = sorted(glob("testing_out/grouped/*_IA*.sc"), reverse=True)
files

In [ ]:
scores_sasa = [col for col in scores.columns if "sasa" in col.lower()]
scores_sasa
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores[scores_sasa].transpose())

In [ ]:
scores_hbond = [col for col in scores.columns if "loop_sc" in col.lower()]
scores_hbond
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores[scores_hbond].transpose())

In [ ]:
truncator.score_interface("data/ZCON_37_N3_C1_2.pdb",
                script_name="truncator/xml/04_FD_FR_and_scoring_benchmark_v3_digs_updated_schema_BUNS3_IA_select.xml",
                out_dir="testing_out/grouped", tee=False, test_run=True)

In [ ]:

scores = truncator.read_score_files(['testing_out/grouped/ZCON_37_N3_C1_2_IA.sc'], skiprows=False, load_seq=False).set_index('description')
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 80, "display.width", 50):
    display(scores.transpose())

In [ ]:
interesting_cols = """
interface_averageDegree_chA
interface_averageDegree_chB
interface_buried_sasa
interface_contactcount
interface_fxn_hydrophobic
interface_hbonds
interface_holes
interface_hydrophobic_sasa
interface_polar_sasa
interface_salt_bridges
interface_sc
interface_sc_int_area
interface_unsat_hbond1
interface_unsat_hbond2
cross_interface_energy_IA
cross_interface_hbonds_IA
delta_unsat_hbonds_IA
hbond_energy_IA
interface_delta_sasa_IA
interface_hydrophobic_sasa_IA
interface_polar_sasa_IA
exposed_hydrophobics
exposed_polars
exposed_total
""".split()
#print("\n".join(list(df.columns)))
df[interesting_cols].transpose()

In [ ]:
import os
import subprocess, sys
os.system("echo TEST")
!ls -l

In [ ]:
pd.set_option('display.max_rows', None)
pd.options.display.max_columns=None
df.transpose()

In [ ]:
import dask
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(cores=1,
                     memory="2GB",
                     queue='interactive',)
cluster.scale(4)

from dask.distributed import Client
client = Client(cluster)